In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/train.csv')
val_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/val.csv')
test_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/test.csv')

dirty_train_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/dirty_train.csv')
dirty_val_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/dirty_val.csv')
dirty_test_df = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/dirty_test.csv')


In [3]:
print(len(train_df) + len(val_df) + len(test_df))
print(len(dirty_train_df) + len(dirty_val_df) + len(dirty_test_df))

223549
192960


In [42]:
def combine_and_split_two(df1, df2, base_path):
    # Combine the two dataframes into one
    df = pd.concat([df1, df2], ignore_index=True)

    # Split the combined dataframe into train, validation, and test sets
    train, test = train_test_split(df, test_size=0.15, random_state=42)
    train, val = train_test_split(train, test_size=0.1765, random_state=42)

    # Save the resulting dataframes as CSV files
    train.to_csv(base_path + 'train.csv', index=False)
    val.to_csv(base_path + 'val.csv', index=False)
    test.to_csv(base_path + 'test.csv', index=False)

In [43]:
def split_one(df, base_path):
    # Split the combined dataframe into train, validation, and test sets
    train, test = train_test_split(df, test_size=0.15, random_state=42)
    train, val = train_test_split(train, test_size=0.1765, random_state=42)

    # Save the resulting dataframes as CSV files
    train.to_csv(base_path + 'train.csv', index=False)
    val.to_csv(base_path + 'val.csv', index=False)
    test.to_csv(base_path + 'test.csv', index=False)

split_one(dirty_df, '/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/jigsaw_data/jigsaw-toxic-comment-classification-challenge/dirty_')